In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [6]:
train= pd.read_csv('./train_temp.txt', sep="\t", header=None)
test = pd.read_csv('./test_temp.txt', sep="\t", header=None)

In [7]:
data = pd.concat([train,test], ignore_index=True)
data.columns=['Word','Tag']

In [8]:
data.head()

,Word,Tag
0,Fossavatnsgangan,B-Miscellaneous
1,á,O
2,Ísafirði,B-Location
3,er,O
4,elsta,O


In [9]:
from Malfong_datasets.labels_sentence_num import add_sentence_id
data2 = add_sentence_id(data)

In [10]:
data2.to_pickle('./Malfong_datasets/all_files_with_sen_num.pkl') 

In [11]:
data2.head()

,Word,Tag,Sentence
0,Fossavatnsgangan,B-Miscellaneous,0
1,á,O,0
2,Ísafirði,B-Location,0
3,er,O,0
4,elsta,O,0


In [17]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(), s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [18]:
getter = SentenceGetter(data2)
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
labels = [[s[1] for s in sentence] for sentence in getter.sentences]

In [19]:
sentences[0]

['Fossavatnsgangan',
 'á',
 'Ísafirði',
 'er',
 'elsta',
 'skíðamót',
 'sem',
 'enn',
 'er',
 'við',
 'lýði',
 'á',
 'Íslandi',
 'og',
 'hefur',
 'jafnframt',
 'verið',
 'fjölmennasta',
 'skíðagöngumót',
 'landsins',
 'undanfarin',
 'ár',
 ',',
 'með',
 '100-150',
 'þátttakendur',
 '.']

In [20]:
print(labels[0])

['B-Miscellaneous', 'O', 'B-Location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [21]:
tag_values = list(set(data2["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [24]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.6.0'

In [25]:
MAX_LEN = 75
bs = 32

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count() # 0

In [41]:
from transformers import AutoTokenizer

In [47]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer("/home/elena/Projects/neurocode/icelandic-language-model/src/data/icelandic/final/vocab.json",
    "/home/elena/Projects/neurocode/icelandic-language-model/src/data/icelandic/final/merges.txt")

In [51]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=50)

In [52]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.encode(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [53]:
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sent, labs) for sent, labs in zip(sentences, labels)]

TypeError: 'tokenizers.Encoding' object is not iterable